In [8]:
from ECC_inc import *

In [3]:
fp_print(param.p)

1A0111EA397FE69A 4B1BA7B6434BACD7 64774B84F38512BF 6730D2A0F6B0F624 1EABFFFEB153FFFF B9FEFFFFFFFFAAAB


In [3]:
import random

# Def

In [1]:
# ===== constants.py =====
import random

def bits_of(k):
    return [int(c) for c in "{0:b}".format(k)]

def is_qr(a: int, p: int):
    p_ = (p-1) >> 1
    x = pow(a, p_, p)
    return x == 1

# BLS12-381

class Fp:
    def __init__(self, value) -> None:
        self.p = 0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab
        self.value = value
    
    def __add__(self, other):
        result = (self.value + other.value) % self.p
        return Fp(result)
    
    def __sub__(self, other):
        result = (self.value - other.value) % self.p
        return Fp(result)
    
    def __mul__(self, other):
        result = (self.value * other.value) % self.p
        return Fp(result)
    
    def inv(self):
        result = pow(self.value, (self.p - 2), self.p)
        return Fp(result)
    
    def __truediv__(self, other):
        other_inv = other.inv()
        return self * other_inv
    
    def __pow__(self, exponent: int):
        result = pow(self.value, exponent, self.p)
        return Fp(result)
    
    # reference: https://zenn.dev/peria/articles/c6afc72b6b003c
    def sqrt(self):
        if not is_qr(self.value, self.p):
            return Fp(0), False
        pmod8 = self.p % 8
        if pmod8 == 3 or pmod8 == 7:
            res = self ** ((self.p + 1) >> 2)
            return res, True
        if pmod8 == 5:
            res = self ** ((self.p + 3) >> 3)
            if res ** 2 == self:
                res *= Fp(2) ** ((self.p - 1) >> 2)
            return res, True
        if pmod8 == 1:
            z = random.randint(0, self.p - 1)
            while is_qr(z, self.p):
                z = random.randint(0, self.p - 1)
            z = Fp(z)
            t = self.p - 1
            s = 0
            while t % 2 == 0:
                t >>= 1
                s += 1
            A = self ** t
            D = z ** t
            m = 0
            for i in range(s):
                tmp = (A * (D ** m)) ** (2 ** (s - 1 - i))
                if tmp.value == self.p-1:
                    m = m + (2 ** i)
            res = (self ** ((t + 1) // 2)) * (D ** (m // 2))
            return res, True
        return Fp(0), False
    
    def sign(self):
        if self.value > ((self.p - 1) / 2):
            return Fp(-1)
        return Fp(1)

u = -(2**63 + 2**62 + 2**60 + 2**57 + 2**48 + 2**16)
r = u**4 - u**2 + 1
p = (((u-1)**2) * r) // 3 + u
h1 = ((u-1)**2) // 3 
h2 = (u**8 - 4*(u**7) + 5*(u**6) - 4*(u**4) + 6*(u**3) - 4*(u**2) - 4*u + 13)//9
    
a =  Fp(0) 
b = Fp(4)
A_ =  Fp(0x144698a3b8e9433d693a02c96d4982b0ea985383ee66a8d8e8981aefd881ac98936f8da0e0f97f5cf428082d584c1d) 
B_ = Fp(0x12e2908d11688030018b12e8753eee3b2016c1f0f24f4070a0b9c14fcef35ef55a23215a316ceaa5d1cc48e98e172be0)
xi = Fp(-1)
zero = Fp(0)
one = Fp(1)
k_ = [[0x11a05f2b1e833340b809101dd99815856b303e88a2d7005ff2627b56cdb4e2c85610c2d5f2e62d6eaeac1662734649b7,
0x17294ed3e943ab2f0588bab22147a81c7c17e75b2f6a8417f565e33c70d1e86b4838f2a6f318c356e834eef1b3cb83bb,
0xd54005db97678ec1d1048c5d10a9a1bce032473295983e56878e501ec68e25c958c3e3d2a09729fe0179f9dac9edcb0,
0x1778e7166fcc6db74e0609d307e55412d7f5e4656a8dbf25f1b33289f1b330835336e25ce3107193c5b388641d9b6861,
0xe99726a3199f4436642b4b3e4118e5499db995a1257fb3f086eeb65982fac18985a286f301e77c451154ce9ac8895d9,
0x1630c3250d7313ff01d1201bf7a74ab5db3cb17dd952799b9ed3ab9097e68f90a0870d2dcae73d19cd13c1c66f652983,
0xd6ed6553fe44d296a3726c38ae652bfb11586264f0f8ce19008e218f9c86b2a8da25128c1052ecaddd7f225a139ed84,
0x17b81e7701abdbe2e8743884d1117e53356de5ab275b4db1a682c62ef0f2753339b7c8f8c8f475af9ccb5618e3f0c88e,
0x80d3cf1f9a78fc47b90b33563be990dc43b756ce79f5574a2c596c928c5d1de4fa295f296b74e956d71986a8497e317,
0x169b1f8e1bcfa7c42e0c37515d138f22dd2ecb803a0c5c99676314baf4bb1b7fa3190b2edc0327797f241067be390c9e,
0x10321da079ce07e272d8ec09d2565b0dfa7dccdde6787f96d50af36003b14866f69b771f8c285decca67df3f1605fb7b,
0x6e08c248e260e70bd1e962381edee3d31d79d7e22c837bc23c0bf1bc24c6b68c24b1b80b64d391fa9c8ba2e8ba2d229
],
[0x8ca8d548cff19ae18b2e62f4bd3fa6f01d5ef4ba35b48ba9c9588617fc8ac62b558d681be343df8993cf9fa40d21b1c,
0x12561a5deb559c4348b4711298e536367041e8ca0cf0800c0126c2588c48bf5713daa8846cb026e9e5c8276ec82b3bff,
0xb2962fe57a3225e8137e629bff2991f6f89416f5a718cd1fca64e00b11aceacd6a3d0967c94fedcfcc239ba5cb83e19,
0x3425581a58ae2fec83aafef7c40eb545b08243f16b1655154cca8abc28d6fd04976d5243eecf5c4130de8938dc62cd8,
0x13a8e162022914a80a6f1d5f43e7a07dffdfc759a12062bb8d6b44e833b306da9bd29ba81f35781d539d395b3532a21e,
0xe7355f8e4e667b955390f7f0506c6e9395735e9ce9cad4d0a43bcef24b8982f7400d24bc4228f11c02df9a29f6304a5,
0x772caacf16936190f3e0c63e0596721570f5799af53a1894e2e073062aede9cea73b3538f0de06cec2574496ee84a3a,
0x14a7ac2a9d64a8b230b3f5b074cf01996e7f63c21bca68a81996e1cdf9822c580fa5b9489d11e2d311f7d99bbdcc5a5e,
0xa10ecf6ada54f825e920b3dafc7a3cce07f8d1d7161366b74100da67f39883503826692abba43704776ec3a79a1d641,
0x95fc13ab9e92ad4476d6e3eb3a56680f682b4ee96f7d03776df533978f31c1593174e4b4b7865002d6384d168ecdd0a
],
[0x90d97c81ba24ee0259d1f094980dcfa11ad138e48a869522b52af6c956543d3cd0c7aee9b3ba3c2be9845719707bb33,
0x134996a104ee5811d51036d776fb46831223e96c254f383d0f906343eb67ad34d6c56711962fa8bfe097e75a2e41c696,
0xcc786baa966e66f4a384c86a3b49942552e2d658a31ce2c344be4b91400da7d26d521628b00523b8dfe240c72de1f6,
0x1f86376e8981c217898751ad8746757d42aa7b90eeb791c09e4a3ec03251cf9de405aba9ec61deca6355c77b0e5f4cb,
0x8cc03fdefe0ff135caf4fe2a21529c4195536fbe3ce50b879833fd221351adc2ee7f8dc099040a841b6daecf2e8fedb,
0x16603fca40634b6a2211e11db8f0a6a074a7d0d4afadb7bd76505c3d3ad5544e203f6326c95a807299b23ab13633a5f0,
0x4ab0b9bcfac1bbcb2c977d027796b3ce75bb8ca2be184cb5231413c4d634f3747a87ac2460f415ec961f8855fe9d6f2,
0x987c8d5333ab86fde9926bd2ca6c674170a05bfe3bdd81ffd038da6c26c842642f64550fedfe935a15e4ca31870fb29,
0x9fc4018bd96684be88c9e221e4da1bb8f3abd16679dc26c1e8b6e6a1f20cabe69d65201c78607a360370e577bdba587,
0xe1bba7a1186bdb5223abde7ada14a23c42a0ca7915af6fe06985e7ed1e4d43b9b3f7055dd4eba6f2bafaaebca731c30,
0x19713e47937cd1be0dfd0b8f1d43fb93cd2fcbcb6caf493fd1183e416389e61031bf3a5cce3fbafce813711ad011c132,
0x18b46a908f36f6deb918c143fed2edcc523559b8aaf0c2462e6bfe7f911f643249d9cdf41b44d606ce07c8a4d0074d8e,
0xb182cac101b9399d155096004f53f447aa7b12a3426b08ec02710e807b4633f06c851c1919211f20d4c04f00b971ef8,
0x245a394ad1eca9b72fc00ae7be315dc757b3b080d4c158013e6632d3c40659cc6cf90ad1c232a6442d9d3f5db980133,
0x5c129645e44cf1102a159f748c4a3fc5e673d81d7e86568d9ab0f5d396a7ce46ba1049b6579afb7866b1e715475224b,
0x15e6be4e990f03ce4ea50b3b42df2eb5cb181d8f84965a3957add4fa95af01b2b665027efec01c7704b456be69c8b604
],
[0x16112c4c3a9c98b252181140fad0eae9601a6de578980be6eec3232b5be72e7a07f3688ef60c206d01479253b03663c1,
0x1962d75c2381201e1a0cbd6c43c348b885c84ff731c4d59ca4a10356f453e01f78a4260763529e3532f6102c2e49a03d,
0x58df3306640da276faaae7d6e8eb15778c4855551ae7f310c35a5dd279cd2eca6757cd636f96f891e2538b53dbf67f2,
0x16b7d288798e5395f20d23bf89edb4d1d115c5dbddbcd30e123da489e726af41727364f2c28297ada8d26d98445f5416,
0xbe0e079545f43e4b00cc912f8228ddcc6d19c9f0f69bbb0542eda0fc9dec916a20b15dc0fd2ededda39142311a5001d,
0x8d9e5297186db2d9fb266eaac783182b70152c65550d881c5ecd87b6f0f5a6449f38db9dfa9cce202c6477faaf9b7ac,
0x166007c08a99db2fc3ba8734ace9824b5eecfdfa8d0cf8ef5dd365bc400a0051d5fa9c01a58b1fb93d1a1399126a775c,
0x16a3ef08be3ea7ea03bcddfabba6ff6ee5a4375efa1f4fd7feb34fd206357132b920f5b00801dee460ee415a15812ed9,
0x1866c8ed336c61231a1be54fd1d74cc4f9fb0ce4c6af5920abc5750c4bf39b4852cfe2f7bb9248836b233d9d55535d4a,
0x167a55cda70a6e1cea820597d94a84903216f763e13d87bb5308592e7ea7d4fbc7385ea3d529b35e346ef48bb8913f55,
0x4d2f259eea405bd48f010a01ad2911d9c6dd039bb61a6290e591b36e636a5c871a5c29f4f83060400f8b49cba8f6aa8,
0xaccbb67481d033ff5852c1e48c50c477f94ff8aefce42d28c0f9a88cea7913516f968986f7ebbea9684b529e2561092,
0xad6b9514c767fe3c3613144b45f1496543346d98adf02267d5ceef9a00d9b8693000763e3b90ac11e99b138573345cc,
0x2660400eb2e4f3b628bdd0d53cd76f2bf565b94e72927c1cb748df27942480e420517bd8714cc80d1fadc1326ed06f7,
0xe0fa1d816ddc03e6b24255e0d7819c171c40f65e273b853324efcd6356caa205ca2f570f13497804415473a1d634b8f
]]
k = [[Fp(k_[i][j]) for j in range(len(k_[i]))] for i in range(len(k_))]

    
class pointFp:
    def __init__(self, coord_type, coordinate: list[Fp], coefficients: list[Fp]):
        self.p = 0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab
        self.coord_type = coord_type
        self.X = coordinate[0]
        self.Y = coordinate[1]
        self.Z = coordinate[2]
        self.a = coefficients[0]
        self.b = coefficients[1]
        self.b3 = Fp(3) * self.b
        
    # For projective coordinate
    # Reference: 
        # Renes, Joost, Craig Costello, and Lejla Batina. 
        # "Complete addition formulas for prime order elliptic curves." 
        # Advances in Cryptology–EUROCRYPT 2016: 35th Annual International Conference on the Theory and Applications of Cryptographic Techniques, 
        # Vienna, Austria, May 8-12, 2016.
    # Computation cost: 8M + 3S + 3m_a + 2m_3b + 15a (for the case a != 0)
    def __add__(self, other):
        assert self.a == other.a and self.b == other.b, "their curves are different"
        assert self.coord_type == "projective", "the coordinate type is %s" % self.coord_type

        t0 = self.X * other.X
        t1 = self.Y * other.Y
        t2 = self.Z * other.Z
        t3 = self.X + self.Y
        t4 = other.X + other.Y
        t3 = t3 * t4
        t4 = t0 + t1
        t3 = t3 - t4
        t4 = self.X + self.Z
        t5 = other.X + other.Z
        t4 = t4 * t5
        t5 = t0 + t2
        t4 = t4 - t5
        t5 = self.Y + self.Z
        X3 = other.Y + other.Z
        t5 = t5 * X3
        X3 = t1 + t2
        t5 = t5 - X3
        Z3 = self.a * t4
        X3 = self.b3 * t2
        Z3 = X3 + Z3
        X3 = t1 - Z3
        Z3 = t1 + Z3
        Y3 = X3 * Z3
        t1 = t0 + t0
        t1 = t1 + t0
        t2 = self.a * t2
        t4 = self.b3 * t4
        t1 = t1 + t2
        t2 = t0 - t2
        t2 = self.a * t2
        t4 = t4 + t2
        t0 = t1 * t4
        Y3 = Y3 + t0
        t0 = t5 * t4
        X3 = t3 * X3
        X3 = X3 - t0
        t0 = t3 * t1
        Z3 = t5 * Z3
        Z3 = Z3 + t0
        return pointFp(coord_type="projective", coordinate=[X3, Y3, Z3], coefficients=[self.a, self.b])
        
    def double(self):
        t0 = self.X * self.X
        t1 = self.Y * self.Y
        t2 = self.Z * self.Z
        t3 = self.X * self.Y
        t3 = t3 + t3
        Z3 = self.X * self.Z
        Z3 = Z3 + Z3
        X3 = self.a * Z3
        Y3 = self.b3 * t2
        Y3 = X3 + Y3
        X3 = t1 - Y3
        Y3 = t1 + Y3
        Y3 = X3 * Y3
        X3 = t3 * X3
        Z3 = self.b3 * Z3
        t2 = self.a * t2
        t3 = t0 - t2
        t3 = self.a * t3
        t3 = t3 + Z3
        Z3 = t0 + t0
        t0 = Z3 + t0
        t0 = t0 + t2
        t0 = t0 * t3
        Y3 = Y3 + t0
        t2 = self.Y * self.Z
        t2 = t2 + t2
        t0 = t2 * t3
        X3 = X3 - t0
        Z3 = t2 * t1
        Z3 = Z3 + Z3
        Z3 = Z3 + Z3
        return pointFp(coord_type="projective", coordinate=[X3, Y3, Z3], coefficients=[self.a, self.b])
    
    def scalar_mul(self, n):
        res = self
        for nb in bits_of(n)[1:]:
            res = self.double()
            if(nb == 1):
                res = res + self
        return res
        
    def check_on_curve(self):
        if self.coord_type == "projective":
            X_ = self.X / self.Z
            Y_ = self.Y / self.Z
            gx = X_ ** 3 + self.a * X_ + self.b
            y2 = Y_ ** 2
            if gx.value == y2.value:
                print("on curve")
            else:
                print("\ng(x) = %x\ny    = %x" % (gx.value, y2.value))
        else:
            raise Exception("not implemented")
    
    
    def check_rP_is_inf(self):
        tmp = self
        # if self.coord_type != "affine":
        #     tmp = pointFp(coord_type="affine", coordinate=[self.X / self.Z, self.Y / self.Z, Fp(1)], coefficients=[self.a, self.b])
        inf = tmp.scalar_mul(r)
        if inf.Z.value == 0:
            return True
        print(inf.X.value, inf.Y.value, inf.Z.value)
        return False
        
def random_pointFp(p: int, coefficients: list[Fp]):
    sqrtIsExist = False
    while not sqrtIsExist:
        X3 = Fp(random.randint(0, p-1))
        Y3, sqrtIsExist = (X3**3 + coefficients[0]*X3 + coefficients[1]).sqrt()
    res = pointFp(coord_type="projective", coordinate=[X3, Y3, Fp(1)], coefficients=coefficients)
    res.check_on_curve()
    hashed_point = res.scalar_mul(h1)
    hashed_point.check_on_curve()
    # TODO: check if rP is infinity point
    return hashed_point
        
random_point = random_pointFp(p, [a, b])

on curve
on curve


In [30]:
# ===== sha256.py =====
import hashlib

####### operations
def SHR(x, n):
    return x >> n

def ROTR(x, n):
    return (x >> n) | (x << (32 - n))

def ROTL(x, n):
    return (x << n) | (x >> (32 - n))

def Ch(x, y, z):
    return (x & y) ^ (~x & z)

def Maj(x, y, z):
    return (x & y) ^ (x & z) ^ (y & z)

def SIGMA0(x):
    return ROTR(x, 2) ^ ROTR(x, 13) ^ ROTR(x, 22)

def SIGMA1(x):
    return ROTR(x, 6) ^ ROTR(x, 11) ^ ROTR(x, 25)

def sigma0(x):
    return ROTR(x, 7) ^ ROTR(x, 18) ^ SHR(x, 3)

def sigma1(x):
    return ROTR(x, 17) ^ ROTR(x, 19) ^ SHR(x, 10)

####### constants

K = [0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2]

H = [0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a, 0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19]

####### algorithm

# input: a message of any length less than 2^64 bits
def padding(msg: int):
    # print("padding - msg: ", msg)
    msg_len = msg.bit_length()
    msg_len += 4 - (msg_len % 4)
    # print("padding - msg_len: ", msg_len)
    if (msg_len % 512 < 448):
        shift_num = 512 - (msg_len % 512)
        M = (msg << shift_num) + (1 << (shift_num-1)) + msg_len
    else:
        shift_num = 1024 - (msg_len % 512)
        M = (msg << shift_num) + (1 << (shift_num-1)) + msg_len
    # print("padding - shift_num: ", shift_num)
    # print("padding - M: ", "%dbits: %x" % (M.bit_length(), M))
    print("%dbits: %x" % (M.bit_length(), M))
    return M

def hash(M: int):
    N = (M.bit_length() + (512 - (M.bit_length() % 512))) // 512
    # print("%d block" % N)
    print("hash - N: ","%d block" % N)
    H = [0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a, 0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19]
    for i in range(N):
        W = [0] * 64
        Mi = (M >> (512 * (N - i - 1))) & ((1 << 512) - 1)
        print("%x" % Mi)
        for j in range(64):
            if (j < 16):
                W[j] = (Mi >> (32 * (15 - j))) & ((1 << 32) - 1)
            else:
                W[j] = (sigma1(W[j-2]) + W[j-7] + sigma0(W[j-15]) + W[j-16]) & 0xffffffff
        [a, b, c, d, e, f, g, h] = H
        for j in range(64):
            T1 = (h + SIGMA1(e) + Ch(e, f, g) + K[j] + W[j]) & 0xffffffff
            T2 = (SIGMA0(a) + Maj(a, b, c)) & 0xffffffff
            h = g
            g = f
            f = e
            e = (d + T1) & 0xffffffff
            d = c
            c = b
            b = a
            a = (T1 + T2) & 0xffffffff
        H[0] = (a + H[0]) & 0xffffffff         
        H[1] = (b + H[1]) & 0xffffffff         
        H[2] = (c + H[2]) & 0xffffffff         
        H[3] = (d + H[3]) & 0xffffffff         
        H[4] = (e + H[4]) & 0xffffffff         
        H[5] = (f + H[5]) & 0xffffffff         
        H[6] = (g + H[6]) & 0xffffffff         
        H[7] = (h + H[7]) & 0xffffffff  
    res = 0
    for i in range(8):
        res = (res << 32) | H[i]
    return res

def sha256(msg: bytes) -> bytes: 
    msg_as_int = int.from_bytes(msg, 'big')
    pad_msg = padding(msg_as_int)
    # print("sha256 - pad_msg: ", pad_msg)
    hashed_msg = hash(pad_msg)
    # print("sha256 - hashed_msg: ", hashed_msg)
    return hashed_msg.to_bytes(32, 'big')

# if __name__ == "__main__":
# Len = 1
# Msg = 0x451101250ec6f26652249d59dc974b7361d571a8101cdfd36aba3b5854d3ae086b5fdd4597721b66e3c0dc5d8c606d9657d0e323283a5217d1f53f2f284f57b85c8a61ac8924711f895c5ed90ef17745ed2d728abd22a5f7a13479a462d71b56c19a74a40b655c58edfe0a188ad2cf46cbf30524f65d423c837dd1ff2bf462ac4198007345bb44dbb7b1c861298cdf61982a833afc728fae1eda2f87aa2c9480858bec
# MD = 0x3c593aa539fdcdae516cdf2f15000f6634185c88f505b39775fb9ab137a10aa2
# Msg_bytes = Msg.to_bytes(256, 'big')
# MD_bytes = MD.to_bytes(32, 'big')

# res = sha256(Msg_bytes)

# assert res == MD_bytes, "Failed...\nmy_ans = %x\MD     = %s" % (res, MD_bytes)

In [31]:
h_res = sha256("abc".encode('utf-8'))
h_res_int = int.from_bytes(h_res, 'big')
print(hex(h_res_int))


511bits: 61626380000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018
hash - N:  1 block
61626380000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018
0xba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad


In [37]:
len("ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad")

64

In [39]:
len("5396B7D8E5D036A1801B2D1D74B0D076F4F2068719DF7C02BB249338E11BA1E0C3BA23CCE3A24FA0B3D3143935805A48128D1EFE1931AB8BABC3E31E6EA54E379850C7A84427DB916D96CDFC2C71B879A52BE4893A4F469C8CFF072C32DDB7A12308F9D011CAE8A9C07DBAA017EB6EE6C68D05EDE6BB2D4C750AB95A313A4631")

256

In [45]:
# ===== h2f.py =====
import math
# from sha256 import sha256

# expand_message_xmd(msg, DST, len_in_bytes)
# Parameters:
# - H, a hash function (see requirements above).
# - b_in_bytes, b / 8 for b the output size of H in bits.
#  For example, for b = 256, b_in_bytes = 32.
# - s_in_bytes, the input block size of H, measured in bytes (see discussion above). For example, for SHA-256, s_in_bytes = 64.
# Input:
# - msg, a byte string.
# - DST, a byte string of at most 255 bytes.
# - len_in_bytes, the length of the requested output in bytes, not greater than the lesser of (255 * b_in_bytes) or 2^16-1.
# Output:
# - uniform_bytes, a byte string.
# Steps:
# 1. ell = ceil(len_in_bytes / b_in_bytes)
# 2. ABORT if ell > 255 or len_in_bytes > 65535 or len(DST) > 255
# 3. DST_prime = DST || I2OSP(len(DST), 1)
# 4. Z_pad = I2OSP(0, s_in_bytes)
# 5. l_i_b_str = I2OSP(len_in_bytes, 2)
# 6. msg_prime = Z_pad || msg || l_i_b_str || I2OSP(0, 1) || DST_prime
# 7. b_0 = H(msg_prime)
# 8. b_1 = H(b_0 || I2OSP(1, 1) || DST_prime)
# 9. for i in (2, ..., ell):
# 10. b_i = H(strxor(b_0, b_(i - 1)) || I2OSP(i, 1) || DST_prime)
# 11. uniform_bytes = b_1 || ... || b_ell
# 12. return substr(uniform_bytes, 0, len_in_bytes)

b_in_bytes = 32
s_in_bytes = 64


def I2OSP(x: int, xLen: int) -> bytes:
    if x >= 256**xLen:
        raise Exception("I2OSP: integer too large")
    return x.to_bytes(xLen, 'big')

def OS2IP(X: bytes) -> int:
    return int.from_bytes(X, 'big')
    
def strxor(s1: bytes, s2: bytes):    
    # returns the bitwise XOR of the two strings
    # For example, strxor("abc", "XYZ") == "9;9" (ASCII literal)
    return bytes(a ^ b for a, b in zip(s1, s2))

def expand_message(msg: bytes, dst: bytes, len_in_bytes: int) -> bytes:
    ell =  math.ceil(len_in_bytes / b_in_bytes)
    if ell > 255 or len_in_bytes > 65535 or len(dst) > 255:
        raise Exception("The length of the requested output in bytes is greater than the lesser of (255 * b_in_bytes) or 2^16-1")
    dst_prime = dst + I2OSP(len(dst), 1)
    Z_pad = I2OSP(0, s_in_bytes)
    l_i_b_str = I2OSP(len_in_bytes, 2)
    msg_prime = Z_pad + msg + l_i_b_str + I2OSP(0, 1) + dst_prime
    # print("expand_message - msg_prime: ", msg_prime)
    b_0 = sha256(msg_prime)
    # print("expand_message - 2nd sha256 input: ", b_0)
    # print("expand_message - 2nd sha256 input: ", I2OSP(1, 1))
    # print("expand_message - 2nd sha256 input: ", dst_prime)
    b_1 = sha256(b_0 + I2OSP(1, 1) + dst_prime)
    uniform_bytes = b_1
    b_i_1 = b_1
    for i in (2, ell+1):
        b_i = sha256(strxor(b_0, b_i_1) + I2OSP(i, 1) + dst_prime)
        uniform_bytes = uniform_bytes + b_i
        b_i_1 = b_i
    return uniform_bytes[:len_in_bytes]

# hash_to_field(msg, count)
# Parameters:
# - DST, a domain separation tag (see Section 3.1).
# - F, a finite field of characteristic p and order q = p^m.
# - p, the characteristic of F (see immediately above).
# - m, the extension degree of F, m >= 1 (see immediately above).
# - L = ceil((ceil(log2(p)) + k) / 8), where k is the security
#  parameter of the suite (e.g., k = 128).
# - expand_message, a function that expands a byte string and
#  domain separation tag into a uniformly random byte string
#  (see Section 5.3).
# Input:
# - msg, a byte string containing the message to hash.
# - count, the number of elements of F to output.
# Output:
# - (u_0, ..., u_(count - 1)), a list of field elements.
# Steps:
# 1. len_in_bytes = count * m * L
# 2. uniform_bytes = expand_message(msg, DST, len_in_bytes)
# 3. for i in (0, ..., count - 1):
# 4. for j in (0, ..., m - 1):
# 5. elm_offset = L * (j + i * m)
# 6. tv = substr(uniform_bytes, elm_offset, L)
# 7. e_j = OS2IP(tv) mod p
# 8. u_i = (e_0, ..., e_(m - 1))
# 9. return (u_0, ..., u_(count - 1))


##### for BLS12-381
xx = "01"               # two-digit numbers indicating the version?
yy = "02"               # two-digit numbers indicating the version?
CURVE_ID = "BLS12381G1" # a human-readable representation of the target elliptic curve. 
EXP_TAG = "XMD"         # expand_message_xmd
HASH_NAME = "SHA-256"   # a human-readable name for the underlying hash primitive. 
HASH_ID = "{}:{}".format(EXP_TAG, HASH_NAME)    # a human-readable representation of the expand_message function and any underlying hash primitives used in hash_to_feld
MAP_ID = "SSWU"         # a human-readable representation of the map_to_curve function. Simplified SWU
ENC_VAR = "RO"          # a string indicating the encoding type and other information. hash_to_curve
suiteID = "{}_{}_{}_{}_".format(CURVE_ID, HASH_ID, MAP_ID, ENC_VAR)
DST = "QUUX-V{}-CS{}-{}".format(xx, yy, suiteID)
p = 0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabffeb153fffb9feffffaaab 
m = 1   # the extension degree of field that curve is defined
L = 64

def hash_to_field(msg: bytes, count: int) -> list[list[int]]:
    len_in_bytes = count * m * L
    uniform_bytes = expand_message(msg, DST.encode('utf-8'), len_in_bytes)
    u = [[0] * m for i in range(count)]
    for i in range(count):
        e = [0] * m
        for j in range(m):
            elm_offset = L * (j + i * m)
            tv = uniform_bytes[elm_offset: elm_offset+L]
            print("OS2IP(tv):",i,j,hex(OS2IP(tv)))
            e[j] = OS2IP(tv) % p
        u[i] = e
    return u

# if __name__ == "__main__":
# print(hash_to_field("abc".encode('utf-8'), 2))

In [46]:
len("6ea751d6cf9c285327431a6f708b5cd841fe22dbed5364c3171345f1c16eeadf74b08bc41b5b203dcf7fd902045bbde4c5eb3eecfe9d90d80655e59b3e10ac50")

128

In [48]:
len("5396B7D8E5D036A1801B2D1D74B0D076F4F2068719DF7C02BB249338E11BA1E0C3BA23CCE3A24FA0B3D3143935805A48128D1EFE1931AB8BABC3E31E6EA54E379850C7A84427DB916D96CDFC2C71B879A52BE4893A4F469C8CFF072C32DDB7A12308F9D011CAE8A9C07DBAA017EB6EE6C68D05EDE6BB2D4C750AB95A313A4631")

256

In [47]:
print(hash_to_field("abc".encode('utf-8'), 2))

511bits: 616263008000515555582d5630312d435330322d424c53313233383147315f584d443a5348412d3235365f535357555f524f5f2d8000000000000000000001a0
hash - N:  1 block
616263008000515555582d5630312d435330322d424c53313233383147315f584d443a5348412d3235365f535357555f524f5f2d8000000000000000000001a0
1021bits: 12ba677fc9a57f135c92277c4992a6014f2ebc536459ca152ced276ca2551a9501515555582d5630312d435330322d424c53313233383147315f584d443a5348412d3235365f535357555f524f5f2d80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000278
hash - N:  2 block
12ba677fc9a57f135c92277c4992a6014f2ebc536459ca152ced276ca2551a9501515555582d5630312d435330322d424c53313233383147315f584d443a5348
412d3235365f535357555f524f5f2d80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000278
1023bits: 7c1d36a9063957407bd13d133919fad90ed09e88890aaed63bfe629d633bf04a02515555582d5630312d435330322d424c53313233383147315f584d443a5348412d3235365f535357555f524f5f2d80

In [6]:
print(hash_to_field("abc".encode('utf-8'), 2))

511bits: 616263008000515555582d5630312d435330322d424c53313233383147315f584d443a5348412d3235365f535357555f524f5f2d8000000000000000000001a0
1 block
616263008000515555582d5630312d435330322d424c53313233383147315f584d443a5348412d3235365f535357555f524f5f2d8000000000000000000001a0
1021bits: 12ba677fc9a57f135c92277c4992a6014f2ebc536459ca152ced276ca2551a9501515555582d5630312d435330322d424c53313233383147315f584d443a5348412d3235365f535357555f524f5f2d80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000278
2 block
12ba677fc9a57f135c92277c4992a6014f2ebc536459ca152ced276ca2551a9501515555582d5630312d435330322d424c53313233383147315f584d443a5348
412d3235365f535357555f524f5f2d80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000278
1023bits: 7c1d36a9063957407bd13d133919fad90ed09e88890aaed63bfe629d633bf04a02515555582d5630312d435330322d424c53313233383147315f584d443a5348412d3235365f535357555f524f5f2d800000000000000000000000

In [9]:
DST

'QUUX-V01-CS02-BLS12381G1_XMD:SHA-256_SSWU_RO_'

In [9]:
# ===== h2f.py =====
# from constants import *
# from h2f import hash_to_field

# map_to_curve(u)
# Input: u, an element of field F.
# Output: Q, a point on the elliptic curve E.
# Steps: defined in Section 6.


# Simplified SWU for AB == 0:
# 1. (x', y') = map_to_curve_simple_swu(u) # (x', y') is on E'
# 2. (x, y) = iso_map(x', y') # (x, y) is on E
# 3. return (x, y)

# map_to_curve_simple_swu algorithm is referenced from chapter 4.2 of 
# Wahby, R. S., & Boneh, D. (2019). "Fast and simple constant-time hashing
# to the BLS12-381 elliptic curve". Cryptology ePrint Archive.

def SSWU_before_isogeny(t_: int) -> pointFp:
    if t_ == 0 or t_ == 1 or t_ == -1:
        raise Exception("value t is %d" % t)
    t = Fp(t_)

    t2 = t * t
    xit2 = xi * t2
    xi2t4 = xit2 * xit2
    D_a_ = xi2t4 + xit2
    D_ = D_a_ * A_
    Z = zero - D_
    N_b = D_a_ + one
    X = B_ * N_b
    if Z == zero:
        Z = xi * A_

    D2 = Z * Z
    D3 = D2 * Z
    N2 = X * X
    N3 = N2 * X
    ND2 = X * D2
    aND2 = A_ * ND2
    bD3 = B_ * D3
    U = N3 + aND2 + bD3
    V = D3

    UV = U * V
    V2 = V * V
    UV3 = UV * V2
    UV3_exp = UV3 ** ((UV3.p - 3) // 4)

    alpha = UV3_exp * UV

    alphaD = alpha * Z

    alpha2 = alpha * alpha
    alpha2V = alpha2 * V
    if alpha2V == U:
        Y = t.sign() * alphaD
    else:
        X = xit2 * X
        t3 = t * t2
        Y = t3 * alphaD

    Q = pointFp(coord_type="projective", coordinate=[X, Y, Z], coefficients=[A_, B_])
    Q.check_on_curve()
    return Q


def isogeny(point: pointFp) -> pointFp:
    X = point.X
    Y = point.Y
    Z = point.Z
    # computing Nx
    Nx = zero
    Dx = zero
    Ny = zero
    Dy = zero
    for i in range(len(k[0])):  # 12
        Nx = Nx + k[0][i] * (Z ** (11 - i)) * (X**i)
    for i in range(len(k[1])):  # 10
        Dx = Dx + k[1][i] * (Z ** (10 - i)) * (X**i)
    Dx = Dx + X**10
    Dx = Dx * Z
    for i in range(len(k[2])):  # 16
        Ny = Ny + k[2][i] * (Z ** (15 - i)) * (X**i)
    Ny = Ny * Y
    for i in range(len(k[3])):  # 15
        Dy = Dy + k[3][i] * (Z ** (15 - i)) * (X**i)
    Dy = Dy + X**15
    Dy = Dy * Z

    # return values in Jacobian coordinates
    Z0 = Dx * Dy
    X0 = Nx * Dy
    Y0 = Ny * Dx
    P = pointFp(coord_type="projective", coordinate=[X0, Y0, Z0], coefficients=[a, b])
    P.check_on_curve()
    return P


# scalar mul *h (h: cofactor)
def cofactor_clearing(point: pointFp) -> pointFp:
    return point.scalar_mul(h1)


def map_to_curve_BLS12381G1(t_list: list[list[int]]) -> pointFp:
    Q0 = SSWU_before_isogeny(t_list[0][0])
    Q1 = SSWU_before_isogeny(t_list[1][0])
    Q = Q0 + Q1
    Q.check_on_curve()
    P = isogeny(Q)
    M = cofactor_clearing(P)
    return M


# if __name__ == "__main__":
t_list = hash_to_field("abc".encode("utf-8"), 2)
M = map_to_curve_BLS12381G1(t_list)
M.check_on_curve()
# t_list = hash_to_field("abc".encode("utf-8"), 2)
# Q0 = SSWU_before_isogeny(t_list[0][0])
# Q1 = SSWU_before_isogeny(t_list[1][0])
# Q = Q0 + Q1
# P = isogeny(Q)

print()
print(hex(M.X.value))
print(hex(M.Y.value))
print(hex(M.Z.value))

M_res = ep_t()
M_res.x= M.X.value
M_res.y= M.Y.value
M_res.z= M.Z.value

M_res = ep_norm(M_res)
print()
ep_print(M_res)

511bits: 616263008000515555582d5630312d435330322d424c53313233383147315f584d443a5348412d3235365f535357555f524f5f2d8000000000000000000001a0
1 block
616263008000515555582d5630312d435330322d424c53313233383147315f584d443a5348412d3235365f535357555f524f5f2d8000000000000000000001a0
1021bits: 12ba677fc9a57f135c92277c4992a6014f2ebc536459ca152ced276ca2551a9501515555582d5630312d435330322d424c53313233383147315f584d443a5348412d3235365f535357555f524f5f2d80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000278
2 block
12ba677fc9a57f135c92277c4992a6014f2ebc536459ca152ced276ca2551a9501515555582d5630312d435330322d424c53313233383147315f584d443a5348
412d3235365f535357555f524f5f2d80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000278
1023bits: 7c1d36a9063957407bd13d133919fad90ed09e88890aaed63bfe629d633bf04a02515555582d5630312d435330322d424c53313233383147315f584d443a5348412d3235365f535357555f524f5f2d800000000000000000000000

# Work

In [39]:
t_list = hash_to_field("abc".encode("utf-8"), 2)
M = map_to_curve_BLS12381G1(t_list)
M.check_on_curve()
# t_list = hash_to_field("abc".encode("utf-8"), 2)
# Q0 = SSWU_before_isogeny(t_list[0][0])
# Q1 = SSWU_before_isogeny(t_list[1][0])
# Q = Q0 + Q1
# P = isogeny(Q)

print()
print(hex(M.X.value))
print(hex(M.Y.value))
print(hex(M.Z.value))

M_res = ep_t()
M_res.x= M.X.value
M_res.y= M.Y.value
M_res.z= M.Z.value

M_res = ep_norm(M_res)
print()
ep_print(M_res)

padding - msg:  64376488084852866674797590917394108028152260722021317531805866269204019748262744603465415614159881729123761934133040738426669
hash - N:  1 block
padding - msg:  1303803019916047964325133884252360023724237175853540222919268392028893147433230359436790874059232606646123046396651301843420880529168091048341232086146465728966300698537473955529105651752749
hash - N:  2 block
padding - msg:  2785286452006007373289725773116605497241889015820351147608474181002877698539621456003284402714450922573467595131084594485971537114501856925322308585712321570040753078623864271274833704083245
hash - N:  2 block
padding - msg:  2785286452006007373289725773116605497241889015820351147608474181002877698539623259683987973018369941694565929341921619010169098704162560124788445322243609589325333199454623773638762230144813
hash - N:  2 block

g(x) = 37f881f927b4e16e6e85a748c36e11ad440dee206ed9aa6f00fbdc6332d6345fa8a59a29c01162fd645a4b134341bb8
y    = 168189caa704988364334d41b714cbbc90366ca2ec97781877